In [104]:
import socket
import random
import numpy as np
import matplotlib.pyplot as plt
import time

In [159]:
image = []

with open("umdlogo.jpg", "rb") as jpg:
    
    while True:
        image = jpg.read()
        break
        # byteLine = jpg.readline()
        # if byteLine:

        #     image.append(byteLine)
        # else:
        #     break

In [352]:
test = "hello world!"
empty = ""
for character in test:
    empty += f"{(ord(character)):08b}"

In [354]:
int(empty,base=2)

32309054545037006034346730529

In [355]:
def strToBin(msg: str):
    binRep = ""

    for character in msg:
        binRep += f"{(ord(character)):08b}"

    return binRep

In [362]:
int(strToBin("hello")[0:4], base=2)

6

In [574]:
def separateData(packet):
    stringPacket = [f"{packByte:08b}" for packByte in packet]
    flags = stringPacket[0]
    seqNum = int("".join(stringPacket[1:5]), base = 2)
    ackNum = int("".join(stringPacket[5:9]), base = 2)
    payload = "".join(stringPacket[9:-1])
    return (flags, seqNum, ackNum, payload)


In [563]:
def getCRC(binCode, divisor):
    payload = list(binCode) + ([0]*(len(divisor)-1))
    payload = [int(n) for n in payload]
    divisor = [int(n) for n in list(divisor)]
    remainder = []
    pointer = 0

    while pointer <= len(payload) - len(divisor):
        #Extract slice to subtract
        val = payload[pointer: pointer + len(divisor)]
        
        #Subtract paired values
        operations = zip(val, divisor)
        remainder = [abs(op[0] - op[1]) for op in operations]
        
        #Modify payload values, set pointer to index
        for i in range(len(divisor)):
            payload[pointer + i] = remainder[i]
        
        #Move pointer to next non-zero value
        pointer = payload.index(1)
    return "".join([str(val) for val in remainder])[1:]

In [569]:
def checkCRC(binCode, divisor):
    stringPacket = [f"{packByte:08b}" for packByte in binCode]
    payload = []
    for byteSection in stringPacket:
        payload += byteSection
    payload = [int(n) for n in payload]
    divisor = [int(n) for n in list(divisor)]
    remainder = []
    pointer = 0

    while pointer <= len(payload) - len(divisor):
        #Extract slice to subtract
        val = payload[pointer: pointer + len(divisor)]
        
        #Subtract paired values
        operations = zip(val, divisor)
        remainder = [abs(op[0] - op[1]) for op in operations]
        
        #Modify payload values, set pointer to index
        for i in range(len(divisor)):
            payload[pointer + i] = remainder[i]
        
        #Move pointer to next non-zero value
        if 1 not in payload:
            return 0
        pointer = payload.index(1)

    return int("".join([str(val) for val in remainder])[1:])

In [565]:
def buildPacket(sync = 0, ack = 0, fin = 0, seqNum = 0, ackNum = 0, payload = 0, divisor = "000000000"):
    flags = f"000{ack}00{sync}{fin}"
    stringPacket = flags + f"{seqNum:032b}" + f"{ackNum:032b}" + f"{payload:01024b}"
    CRC = ""
    if int(divisor) > 0:
        CRC = getCRC(stringPacket, divisor)
    else:
        CRC = "00000000"
    stringPacket += CRC
    
    listPacket = np.array([val for val in stringPacket]).reshape(int(np.ceil(len(stringPacket)/8)),8)
    intPacket = [int("".join(listByte), base = 2) for listByte in listPacket]

    return intPacket

In [566]:
def reconstructHeader(packet, low, high):
    headerVal = int("".join([f"{packByte:08b}" for packByte in packet[low:high]]), base = 2)
    return headerVal

In [581]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind(("localhost", 8081))
    s.connect(("localhost", 8001))
    msg_send = input("Enter a message")
    msgBin = strToBin(msg_send)

    # Define some packet info
    seqNum = 0
    ackNum = 0
    divisor = '100000110'

    print(f"Connected to 127")

    # Handshake
    # - Send Sync Request
    syncPacket = bytes(buildPacket(sync=1))
    sent = s.send(syncPacket)

    # - Receive Sync Ack
    ackPacket = s.recv(2048)
    flags, otherSeq, otherAck, otherPayload = separateData(ackPacket)
    ackNum = otherAck
    print(otherAck)

    # - Send Sync Ack
    ackPacket = bytes(buildPacket(ack=1, ackNum=ackNum, seqNum=seqNum))
    seqNum += 1
    sent = s.send(ackPacket)

    while True:
        cnt = 0

        # Send Data
        print("sending payload")
        payloadPiece = bytes(buildPacket(ackNum=ackNum, seqNum=seqNum, payload=int(msgBin[seqNum - 1: seqNum + 1023], base = 2), divisor=divisor))
        sent = s.send(payloadPiece)

        # Wait for acknowledgement
        print("waiting for ack")
        resp = s.recv(2048)
        # print(resp)
        flags, otherSeq, otherAck, otherPayload = separateData(resp)

        seqNum = otherAck

        if seqNum >= len(msg_byte):
            print("Finished sending data")
            s.send(bytes(buildPacket(fin = 1, divisor=divisor)))
            s.close()
            break

        # time.sleep(3)


Connected to 127
1
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
sending payload
waiting for ack
Finished sending data


In [578]:
s.close()

In [560]:
checkCRC(bytes(buildPacket(ackNum=ackNum, seqNum=seqNum, payload=1024, divisor="100000110")), "100000110")

crc [0, 0, 1, 1, 1, 1, 0, 1, 0]
00000000000000000000000000000000000000010000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

ValueError: 1 is not in list